In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import time

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 1000)

## 他のkaggle競技者へ

このノートブックはKaggleを通してスキルを身に付けたい生徒さんに向けて書かれたものの一つです。ここでは主にKaggleで使われる様々なテクニックをデモンストレーションしていきます。多くの人がLB（リーダーボード）を過学習させることに集中してしまいますが、まずは便利な機械学習やCV（分割交差検証）のテクニックを学んで少しずつ積み重ねていきましょう。

チームになって競う生徒さんもいらっしゃるのでカーネルを使って情報を共有することにしました。どうぞ以下でも積極的にコメント等してください。

では楽しんでください！

# 1. Porto Seguo データ概要

このチャレンジでは運転者が保険金請求を行うかどうかを予測する予測モデルを構築します。与えられたデータはすでに処理され、カテゴリカルデータが名前とともに綺麗にラベル付けされています。順不同のカテゴリカルフィーチャもあれば論理的に名付けられた順序値も含まれています。 

データ詳細でも書かれていた通り、
* *bin* はバイナリフィーチャ
* *cat* はカテゴリカルフィーチャ
* これ以外のものは連続的数値変数か順序的定性変数となります。

また、-1という値は欠損値を表しています。

まずは簡単なデータ処理をし、学習およびテストデータのフィーチャがどのようなものなのかまとめてみましょう。

In [3]:
# テストと学習データを取り込み、２つを結合させます。
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
sample_submission=pd.read_csv('../input/sample_submission.csv')

In [4]:
traintest=train.drop(['id','target'], axis=1).append(test.drop(['id'], axis=1))
cols=traintest.columns

# フィーチャ（特徴量）統計のためカラム（縦列）の名前を定義します。
# 名の通りフィーチャから以下の統計を求めます。
# nunique: 一意的な値の個数
# freq1: 出現頻度が最も高い値
# freq1_val: 出現頻度の最も高い値の出現回数
# freq2: 二番目に出現頻度の高い値 
# freq2_val: 二番目に出現頻度の高い値の出現回数
# freq3: 三番目に出現頻度の高い値（あれば）
# freq3_val: 三番目に出現頻度の高い値の出現回数（あれば）
# .describe機能を使った統計も行います。

stat_cols= ['nunique','freq1','freq1_val', 'freq2', 'req2_val',
             'freq3', 'freq3_val'] + traintest[cols[0]].describe().index.tolist()[1:]

stat_cols=['feature']+stat_cols

feature_stat=pd.DataFrame(columns=stat_cols)
i=0

for col in cols:
    stat_vals=[]
    
    # 統計値を求めます。
    stat_vals.append(col)
    stat_vals.append(traintest[col].nunique())
    stat_vals.append(traintest[col].value_counts().index[0])
    stat_vals.append(traintest[col].value_counts().iloc[0])
    stat_vals.append(traintest[col].value_counts().index[1])
    stat_vals.append(traintest[col].value_counts().iloc[1])
    
    if len(traintest[col].value_counts())>2:
        stat_vals.append(traintest[col].value_counts().index[2])
        stat_vals.append(traintest[col].value_counts().iloc[2])
    else:
        stat_vals.append(np.nan)
        stat_vals.append(np.nan)
            
    stat_vals+=traintest[col].describe().tolist()[1:]

    feature_stat.loc[i]=stat_vals
    i+=1

このリストを一意的な値の個数で整理して、カテゴリカルフィーチャを詳しく見てみましょう。これは後で「特別扱い」が必要になってくるものです。

In [5]:
feature_stat[feature_stat['feature'].str.contains("cat")].sort_values(by=['nunique'])

,feature,nunique,freq1,freq1_val,freq2,req2_val,freq3,freq3_val,mean,std,min,25%,50%,75%,max
28,ps_car_08_cat,2,1,1238365,0,249663,NaN,NaN,0.832219,0.373672,0.0,1.0,1.0,1.0,1.0
3,ps_ind_04_cat,3,0,866864,1,620936,-1,228,0.417135,0.493396,-1.0,0.0,0.0,1.0,1.0
22,ps_car_02_cat,3,1,1234979,0,253039,-1,10,0.829937,0.375706,-1.0,1.0,1.0,1.0,1.0
23,ps_car_03_cat,3,-1,1028142,1,276842,0,183044,-0.504896,0.788713,-1.0,-1.0,-1.0,0.0,1.0
25,ps_car_05_cat,3,-1,666910,1,431560,0,389558,-0.158162,0.844506,-1.0,-1.0,0.0,1.0,1.0
27,ps_car_07_cat,3,1,1383070,0,76138,-1,28820,0.910097,0.347212,-1.0,1.0,1.0,1.0,1.0
30,ps_car_10_cat,3,1,1475460,0,12136,2,432,0.992135,0.091565,0.0,1.0,1.0,1.0,2.0
1,ps_ind_02_cat,5,1,1079327,2,309747,3,70172,1.358745,0.663639,-1.0,1.0,1.0,2.0,4.0
29,ps_car_09_cat,6,2,883326,0,486510,1,72947,1.328302,0.978743,-1.0,0.0,2.0,2.0,4.0
4,ps_ind_05_cat,8,0,1319412,6,51877,4,45706,0.406955,1.353300,-1.0,0.0,0.0,0.0,6.0


いくつかあることがわかります。特に ps_car_11_cat のレベル数が高いようです。

# 2. カテゴリカル・フィーチャー・エンコーディング

このデータセットのカテゴリカルフィーチャは既に数値型変換され、運転タイプに関わる様々なクラス（例えばFWDやRWD、4WD）がそれぞれ0,1,2というように数字で表されています。機械学習アルゴリズムの多くは文字列よりも数値データを好むため、数値形式でないデータは予めこのように変換しておくと良いです。

ただし、0,1,2というように、FFから4DWまで運転タイプをランク付ける正しい順序は必ずしも存在しないため、これが返ってまた問題になってしまうこともあります。

順序付けずにエンコーディングする方法として、フリークエンシーエンコーディング（frequency encoding）やバイナリエンコーディング（binary encoding）があります。早速試してみましょう。

## 2.1 フリークエンシー・エンコーディング

フリークエンシー・エンコーディングはカテゴリカルフィーチャのレベルをデータで現れた分で表す方法です。頻出なレベルは大抵同じような性質を持つためこのような操作が行われることが多いですが、最終的にはCV（分割交差検証）を行ってこの操作が本当にプラスをもたらしてくれたのか確認しなければなりません。

まずは一つ便利な機能関数を書いてみましょう。

In [9]:
# 学習用データとテストデータから'cols'を取り出しフリークエンシー（出現頻度）エンコーディングを行います。
def freq_encoding(cols, train_df, test_df):
    # 新しいデータセットを以下のデータセットに保存します。
    result_train_df=pd.DataFrame()
    result_test_df=pd.DataFrame()

    for col in cols:
        
       # 学習用セットの中でフィーチャーの出現頻度をデータフレームとして読み込んでいきます。
        col_freq=col+'_freq'
        freq=train_df[col].value_counts()
        freq=pd.DataFrame(freq)
        freq.reset_index(inplace=True)
        freq.columns=[[col,col_freq]]

        # 'freq'データフレームを学習用データと融合します。
        temp_train_df=pd.merge(train_df[[col]], freq, how='left', on=col)
        temp_train_df.drop([col], axis=1, inplace=True)

        # 'freq'データフレームをテストデータと融合します。
        temp_test_df=pd.merge(test_df[[col]], freq, how='left', on=col)
        temp_test_df.drop([col], axis=1, inplace=True)

        # 学習用セットで現れなかったレベルがテストセットに現れなかった場合、頻度を０と設定します。
        temp_test_df.fillna(0, inplace=True)
        temp_test_df[col_freq]=temp_test_df[col_freq].astype(np.int32)

        if result_train_df.shape[0]==0:
            result_train_df=temp_train_df
            result_test_df=temp_test_df
        else:
            result_train_df=pd.concat([result_train_df, temp_train_df],axis=1)
            result_test_df=pd.concat([result_test_df, temp_test_df],axis=1)
    
    return result_train_df, result_test_df


この関数を私たちのカテゴリカルフィーチャに対して実行してみましょう。

In [7]:
cat_cols=['ps_ind_02_cat','ps_car_04_cat', 'ps_car_09_cat',
          'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_11_cat']

# 学習とテストデータセットのフリークエンシーフィーチャのためのデータフレームの作成
train_freq, test_freq=freq_encoding(cat_cols, train, test)

# 元の学習とテストデータセットに結合させます。
train=pd.concat([train, train_freq], axis=1)
test=pd.concat([test,test_freq], axis=1)

フリークエンシー・フィーチャと元のフィーチャを比べてみましょう。

In [8]:
train[train.columns[train.columns.str.contains('ps_ind_02_cat')]].head(5)

,ps_ind_02_cat,ps_ind_02_cat_freq
0,2,123573
1,1,431859
2,4,11378
3,1,431859
4,2,123573


ps_ind_02_catの出現数が別のレベルに割り当てられました。

変換後は元のフィーチャを取り除きますが、まずはその前にバイナリエンコーディングを行っておきましょう。

## 2.2 バイナリ・エンコーディング（変換）

カテゴリカルエンコーディングの中でもone-hot-codingという方法が頻繁に使われます。各レベルに0ベクトルを割り当て、レベルnのn列目には1を割り当てます。例えば先ほど取り上げた運転タイプFWD,RWD,4WDの場合、FWDは001、RWDは010、4WDは001と表されます。

データを0,1,2と順序付けたくないときに便利な方法ですが、新たに次元の問題が出てきます。もともとレベルがn個あった列をn個のダミー変数列で表しました。特徴量が３つのように少ない場合は問題ありませんが、ps_car_11_catのようにレベル数が多い場合もともと50列しかなかったデータセットに100以上の列を足してしまいます。これを３倍以上大きなデータセットで行なってしまうと[「次元の呪い」](https://ja.wikipedia.org/wiki/%E6%AC%A1%E5%85%83%E3%81%AE%E5%91%AA%E3%81%84)と呼ばれる問題に陥ってしまいます。

KDnuggetsの記事[*"Beyond One-Hot"*](https://www.kdnuggets.com/2015/12/beyond-one-hot-exploration-categorical-variables.html)で上手にまとめられていますが、バイナリエンコーディングはダミー変数列の数を減らし、数値型変換では避けられない順序付けを無くすことができます。カテゴリカルフィーチャから既に数値型に変換されたレベルをバイナリ表記で表すということがポイントです。従って、レベルが４つ（0,1,2,3）あるフィーチャは、0は01、1は01、2は10、3は11、と変換されます。

実際にカテゴリカル変数をバイナリに変換する関数を作成してみましょう。

In [ ]:
# カテゴリカル変数をバイナリに変換する関数を作成します。
# 学習用セットとテストセット、エンコードされるフィーチャーをとり、
# 入力フィーチャをバイナリー表示に変換されたデータセットが２つ返ってきます。
# この関数は符号化されるフィーチャが、nをフィーチャのレベル数としたとき
# 既に0からn-1の範囲で数値型に変換されていることを仮定しています。

def binary_encoding(train_df, test_df, feat):
    # 数値型変換で使用された最大値を計算。
    train_feat_max = train_df[feat].max()
    test_feat_max = test_df[feat].max()
    if train_feat_max > test_feat_max:
        feat_max = train_feat_max
    else:
        feat_max = test_feat_max
        
    # 欠損値にはfeat_max+1を使います。
    train_df.loc[train_df[feat] == -1, feat] = feat_max + 1
    test_df.loc[test_df[feat] == -1, feat] = feat_max + 1
    
    # 有り得るすべてのフィーチャの集合体を作成します。
    union_val = np.union1d(train_df[feat].unique(), test_df[feat].unique())

    # フィーチャから小数点表示で最大値を抜き出します。
    max_dec = union_val.max()
    
    # max_devをバイナリ表示するのに必要な桁数を計算します。
    max_bin_len = len("{0:b}".format(max_dec))
    index = np.arange(len(union_val))
    columns = list([feat])
    
    # フィーチャ全てのレベルを取得するのにバイナリ変換フィーチャ用のデータフレームを作成。
    bin_df = pd.DataFrame(index=index, columns=columns)
    bin_df[feat] = union_val
    
    # フィーチャの各レベルのバイナリ表示を取得。 
    feat_bin = bin_df[feat].apply(lambda x: "{0:b}".format(x).zfill(max_bin_len))
    
    # バイナリ表示を異なる桁数に分割する。
    splitted = feat_bin.apply(lambda x: pd.Series(list(x)).astype(np.uint8))
    splitted.columns = [feat + '_bin_' + str(x) for x in splitted.columns]
    bin_df = bin_df.join(splitted)
    
    # バイナリ変換フィーチャ用のデータフレームを学習用セットとテストセットで結合させ、完成です！  
    train_df = pd.merge(train_df, bin_df, how='left', on=[feat])
    test_df = pd.merge(test_df, bin_df, how='left', on=[feat])
    return train_df, test_df


バイナリ変換関数を実行してみましょう。

In [12]:
cat_cols=['ps_ind_02_cat','ps_car_04_cat', 'ps_car_09_cat',
          'ps_ind_05_cat', 'ps_car_01_cat']

train, test=binary_encoding(train, test, 'ps_ind_02_cat')
train, test=binary_encoding(train, test, 'ps_car_04_cat')
train, test=binary_encoding(train, test, 'ps_car_09_cat')
train, test=binary_encoding(train, test, 'ps_ind_05_cat')
train, test=binary_encoding(train, test, 'ps_car_01_cat')

ps_ind_02_catが含まれている列を全て見てみるとバイナリフィーチャが与えられていることがわかります。

In [13]:
train[train.columns[train.columns.str.contains('ps_ind_02_cat')]].head(5)

,ps_ind_02_cat,ps_ind_02_cat_freq,ps_ind_02_cat_bin_0_x,ps_ind_02_cat_bin_1_x,ps_ind_02_cat_bin_2_x,ps_ind_02_cat_bin_0_y,ps_ind_02_cat_bin_1_y,ps_ind_02_cat_bin_2_y
0,2,123573,0,1,0,0,1,0
1,1,431859,0,0,1,0,0,1
2,4,11378,1,0,0,1,0,0
3,1,431859,0,0,1,0,0,1
4,2,123573,0,1,0,0,1,0


In [14]:
train[train.columns[train.columns.str.contains('ps_car_11_cat')]].head(5)

,ps_car_11_cat,ps_car_11_cat_freq
0,12,7246
1,19,5097
2,60,7992
3,104,85083
4,82,10470


注意深く見てみるとps_car_11_catにはバイナリ変換が行われなかったことがわかります。このカテゴリカルフィーチャには100以上のレベルがあり、実際バイナリ変換を行うと新たに40列以上作成されます。これはone-hotエンコーディングと比べれば小さな削減ではありますが、このデータセットを50列以上から90列以上へと80%拡大させてしまいます。より列数の多い大きなデータセットを扱うときに行っても良いですが、この場合はノイズを増やしてしまうだけなのであまり有利ではありません。

ではこの場合どうしたら良いのでしょうか。[*Triskelion*](https://www.kaggle.com/triskelion)さんが書かれた[*feature engineering*](https://www.slideshare.net/HJvanVeen/feature-engineering-72376750)で取りあげられているトリックに挑戦してみて下さい。k-foldを使ったターゲット・エンコーディングも案ですが、今回ここでは省略させていただき、早速モデリングの方へ進みましょう。

## 3. モデル作成のパイプライン

これまで準備してきた学習およびテストデータを使ってモデルを作成していきましょう。その前にデータセットをもう少し小さくしてみましょう。 [*Heads or Tails*](https://www.kaggle.com/headsortails) が自身のカーネル [*interactive EDA*](https://www.kaggle.com/headsortails/steering-wheel-of-fortune-porto-seguro-eda)でも取り上げてくれたように、calが含まれるフィーチャはあまり便利ではないようなので全て取り除いておきます。

In [ ]:
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train.drop(col_to_drop, axis=1, inplace=True)  
test.drop(col_to_drop, axis=1, inplace=True)  

既にフリークエンシー及びバイナリエンコーディングを行なったため、もとのフィーチャもここで取り除いておきます。ps_car_11_catだけ扱いが違っていたため残しておきます。

In [ ]:
cat_cols=['ps_ind_02_cat','ps_car_04_cat', 'ps_car_09_cat', 'ps_ind_05_cat', 'ps_car_01_cat']
train.drop(cat_cols, axis=1, inplace=True)  
test.drop(cat_cols, axis=1, inplace=True)  

ローカルの学習・テストデータを用意します。

In [ ]:
localtrain, localval=train_test_split(train, test_size=0.25, random_state=2017)

drop_cols=['id','target']
y_localtrain=localtrain['target']
x_localtrain=localtrain.drop(drop_cols, axis=1)

y_localval=localval['target']
x_localval=localval.drop(drop_cols, axis=1)

続いて簡単な機械学習モデルを作成し私たちのデータに対して予測を行なっていきます。ランダムフォレストとロジスティック回帰を使ってみましょう。

## 3.1 ランダムフォレストのモデル作成と予測 
SklearnのランダムフォレストAPIを使って学習を行い、ローカルテストデータに対して予測を行なっていきます。

In [15]:
print('Start training...')
start_time=time.time()

rf=RandomForestClassifier(n_estimators=250, n_jobs=6, min_samples_split=5, max_depth=7,
                          criterion='gini', random_state=0)

rf.fit(x_localtrain, y_localtrain)
rf_valprediction=rf.predict_proba(x_localval)[:,1]

end_time = time.time()
print("it takes %.3f seconds to train and predict" % (end_time - start_time))

Start training...
it takes 25.005 seconds to train and predict


[*競技フォーラム*](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/40368)でも多数取り上げられていますが、AUCを簡単にジニ係数に変換する方法があります。ウィキペディアの[*ジニ係数*](https://ja.wikipedia.org/wiki/%E3%82%B8%E3%83%8B%E4%BF%82%E6%95%B0)でも詳しくまとめられています。

In [16]:
rf_val_auc=roc_auc_score(y_localval, rf_valprediction)
rf_val_gininorm=2*rf_val_auc-1

print('Random Forest Validation AUC is {:.6f}'.format(rf_val_auc))
print('Random Forest Validation Normalised Gini Coefficient is {:.6f}'.format(rf_val_gininorm))

Random Forest Validation AUC is 0.628736
Random Forest Validation Normalised Gini Coefficient is 0.257471


テストデータを使って予測を行いましょう。

In [17]:
x_test=test.drop(['id'], axis=1)
y_testprediction=rf.predict_proba(x_test)[:,1]

rf_submission=sample_submission.copy()
rf_submission['target']=y_testprediction
rf_submission.to_csv('rf_submission.csv', compression='gzip', index=False)

##  3.2 ランダムフォレストのモデル作成と予測

Sklearnを使えばロジスティック回帰の学習と予測はランダムフォレストと似ています。ただしこの場合、まずはフィーチャスケーリング(feature scaling)を行ない、スケーリングされた特徴量を使って学習と予測を行わなければなりません。

In [18]:
scaler = StandardScaler().fit(x_localtrain.values)
x_localtrain_scaled = scaler.transform(x_localtrain)
x_localval_scaled = scaler.transform(x_localval)
x_test_scaled = scaler.transform(x_test)

print('Start training...')
start_time=time.time()

logit=LogisticRegression()
logit.fit(x_localtrain_scaled, y_localtrain)
logit_valprediction=logit.predict_proba(x_localval_scaled)[:,1]

end_time = time.time()
print("it takes %.3f seconds to train and predict" % (end_time - start_time))

Start training...
it takes 7.414 seconds to train and predict


ランダムフォレストと同様、予測を行います。

In [19]:
logit_val_auc=roc_auc_score(y_localval, logit_valprediction)
logit_val_gininorm=2*logit_val_auc-1

print('Logistic Regression Validation AUC is {:.6f}'.format(logit_val_auc))
print('Logistic Regression Validation Normalised Gini Coefficient is {:.6f}'.format(logit_val_gininorm))

Logistic Regression Validation AUC is 0.627760
Logistic Regression Validation Normalised Gini Coefficient is 0.255521


ロジック回帰を使って予測を行なってみましょう。

In [21]:
y_testprediction=logit.predict_proba(x_test_scaled)[:,1]

logit_submission=sample_submission.copy()
logit_submission['target']=y_testprediction
logit_submission.to_csv('logit_submission.csv', compression='gzip', index=False)